In [18]:
for name, module in net.named_modules():
    print(name, module)

 DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
  

In [19]:
# ...existing code...
from ultralytics import YOLO
import torch.nn.utils.prune as prune
import torch.nn as nn

model = YOLO("yolo11n.pt")
net = model.model  # nn.Module

# Robustly identify detection head modules (by name, class or attributes)
head_module_ids = set()
for name, module in net.named_modules():
    lname = name.lower()
    cls_name = module.__class__.__name__.lower()
    # heuristics: name contains 'detect' OR class name contains 'detect'
    # OR module exposes common detect attributes
    if ("detect" in lname) or ("detect" in cls_name) or \
       hasattr(module, "anchors") or hasattr(module, "stride") or \
       hasattr(module, "nl") or hasattr(module, "na") or hasattr(module, "nc"):
        # collect all submodules under this head root
        for sub in module.modules():
            head_module_ids.add(id(sub))
        print(f"Marked detection head root: {name} ({module.__class__.__name__})")

# Also skip modules whose names suggest they belong to the head/prediction layers
SKIP_KEYWORDS = ("detect", "head", "pred", "cls", "reg", "dfl", "anchor")

# 1) Structured channel pruning on Conv2d, but skip head modules
for name, module in net.named_modules():
    if not isinstance(module, nn.Conv2d):
        continue
    # skip explicit head modules
    if id(module) in head_module_ids:
        print(f"Skipping head module {name}")
        continue
    # skip by name heuristics (case-insensitive)
    if any(k in name.lower() for k in SKIP_KEYWORDS):
        print(f"Skipping by name {name}")
        continue
    # safety: skip very small layers
    out_ch = getattr(module, "out_channels", None)
    if out_ch is None or out_ch <= 8:
        print(f"Skipping small/unknown conv {name} (out_channels={out_ch})")
        continue

    try:
        prune.ln_structured(module, name="weight", amount=0.3, n=2, dim=0)
        print(f"Applied structured pruning to {name}")
    except Exception as e:
        print(f"Failed structured prune for {name}: {e}")

# 2) Make pruning permanent (remove reparam wrappers)
for name, module in net.named_modules():
    if isinstance(module, nn.Conv2d) and hasattr(module, "weight_orig"):
        try:
            prune.remove(module, "weight")
            print(f"Removed pruning reparam from {name}")
        except Exception as e:
            print(f"Failed to remove pruning wrapper for {name}: {e}")

#model.save("yolo11n_struc_ch_pruned.pt")
# ...existing code...

Marked detection head root:  (DetectionModel)
Marked detection head root: model.0.conv (Conv2d)
Marked detection head root: model.1.conv (Conv2d)
Marked detection head root: model.2.cv1.conv (Conv2d)
Marked detection head root: model.2.cv2.conv (Conv2d)
Marked detection head root: model.2.m.0.cv1.conv (Conv2d)
Marked detection head root: model.2.m.0.cv2.conv (Conv2d)
Marked detection head root: model.3.conv (Conv2d)
Marked detection head root: model.4.cv1.conv (Conv2d)
Marked detection head root: model.4.cv2.conv (Conv2d)
Marked detection head root: model.4.m.0.cv1.conv (Conv2d)
Marked detection head root: model.4.m.0.cv2.conv (Conv2d)
Marked detection head root: model.5.conv (Conv2d)
Marked detection head root: model.6.cv1.conv (Conv2d)
Marked detection head root: model.6.cv2.conv (Conv2d)
Marked detection head root: model.6.m.0.cv1.conv (Conv2d)
Marked detection head root: model.6.m.0.cv2.conv (Conv2d)
Marked detection head root: model.6.m.0.cv3.conv (Conv2d)
Marked detection head ro

In [16]:
from ultralytics import YOLO
import torch.nn.utils.prune as prune
import torch.nn as nn

# Load pretrained YOLO11n
model = YOLO("yolo11n.pt")
net = model.model         

# Identify detection head root and collect its modules so we never touch them
children = list(net.children())
detect_root = children[-1] if children else None
head_module_ids = set()
if detect_root is not None:
    for m in detect_root.modules():
        head_module_ids.add(id(m))

# Also skip modules whose names suggest they belong to the head/prediction layers
SKIP_KEYWORDS = ("detect", "head", "pred", "cls", "reg", "dfl", "anchor")

for name, module in net.named_modules():
    if not isinstance(module, nn.Conv2d):
        continue
    if id(module) in head_module_ids:
        print(f"Skipping head module {name}")
        continue
    if any(k in name.lower() for k in SKIP_KEYWORDS):
        print(f"Skipping by name {name}")
        continue

    # prune 30% of smallest-magnitude weights (unstructured)
    prune.l1_unstructured(module, name="weight", amount=0.3)
    print(f"Applied unstructured prune to {name}")

#Make pruning permanent (remove reparam wrappers)
for name, module in net.named_modules():
    if isinstance(module, nn.Conv2d) and hasattr(module, "weight_orig"):
        prune.remove(module, "weight")
        print(f"Removed pruning reparam from {name}")

model.save("yolo11n_non_struc_pruned.pt")


Skipping head module model.0.conv
Skipping head module model.1.conv
Skipping head module model.2.cv1.conv
Skipping head module model.2.cv2.conv
Skipping head module model.2.m.0.cv1.conv
Skipping head module model.2.m.0.cv2.conv
Skipping head module model.3.conv
Skipping head module model.4.cv1.conv
Skipping head module model.4.cv2.conv
Skipping head module model.4.m.0.cv1.conv
Skipping head module model.4.m.0.cv2.conv
Skipping head module model.5.conv
Skipping head module model.6.cv1.conv
Skipping head module model.6.cv2.conv
Skipping head module model.6.m.0.cv1.conv
Skipping head module model.6.m.0.cv2.conv
Skipping head module model.6.m.0.cv3.conv
Skipping head module model.6.m.0.m.0.cv1.conv
Skipping head module model.6.m.0.m.0.cv2.conv
Skipping head module model.6.m.0.m.1.cv1.conv
Skipping head module model.6.m.0.m.1.cv2.conv
Skipping head module model.7.conv
Skipping head module model.8.cv1.conv
Skipping head module model.8.cv2.conv
Skipping head module model.8.m.0.cv1.conv
Skippi

Structured (channel) pruning

In [ ]:
model = YOLO("yolo11n.pt")
net = model.model  

# Identify detection head root and collect its modules so we never touch them
children = list(net.children())
detect_root = children[-1] if children else None
head_module_ids = set()
if detect_root is not None:
    for m in detect_root.modules():
        head_module_ids.add(id(m))

# Also skip modules whose names suggest they belong to the head/prediction layers
SKIP_KEYWORDS = ("detect", "head", "pred", "cls", "reg", "dfl", "anchor")

#Skip head modules
for name, module in net.named_modules():
    if not isinstance(module, nn.Conv2d):
        continue
    # skip explicit head modules
    if id(module) in head_module_ids:
        print(f"Skipping head module {name}")
        continue
    # skip by name heuristics
    if any(k in name.lower() for k in SKIP_KEYWORDS):
        print(f"Skipping by name {name}")
        continue

    prune.ln_structured(module, name="weight", amount=0.3, n=2, dim=0) #30% of channels
    print(f"Applied structured pruning to {name}")

#Make pruning permanent (remove reparam wrappers)
for name, module in net.named_modules():
    if isinstance(module, nn.Conv2d) and hasattr(module, "weight_orig"):
        prune.remove(module, "weight")
        print(f"Removed pruning reparam from {name}")


model.save("yolo11n_struc_ch_pruned.pt")

Skipping head module model.0.conv
Skipping head module model.1.conv
Skipping head module model.2.cv1.conv
Skipping head module model.2.cv2.conv
Skipping head module model.2.m.0.cv1.conv
Skipping head module model.2.m.0.cv2.conv
Skipping head module model.3.conv
Skipping head module model.4.cv1.conv
Skipping head module model.4.cv2.conv
Skipping head module model.4.m.0.cv1.conv
Skipping head module model.4.m.0.cv2.conv
Skipping head module model.5.conv
Skipping head module model.6.cv1.conv
Skipping head module model.6.cv2.conv
Skipping head module model.6.m.0.cv1.conv
Skipping head module model.6.m.0.cv2.conv
Skipping head module model.6.m.0.cv3.conv
Skipping head module model.6.m.0.m.0.cv1.conv
Skipping head module model.6.m.0.m.0.cv2.conv
Skipping head module model.6.m.0.m.1.cv1.conv
Skipping head module model.6.m.0.m.1.cv2.conv
Skipping head module model.7.conv
Skipping head module model.8.cv1.conv
Skipping head module model.8.cv2.conv
Skipping head module model.8.m.0.cv1.conv
Skippi